<a href="https://colab.research.google.com/github/Belal-AI/CNN_Model_Transfer_learning/blob/main/Copy_of_Res_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from tqdm import tqdm

In [ ]:
from datasets import load_dataset
from torch.optim import Adam
from tqdm import tqdm
from torch import Tensor
ds = load_dataset("ylecun/mnist")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
import torchvision.transforms as transforms


In [ ]:
!nividia_smi

In [ ]:
import torch
import torch.nn as nn

class Resadual_Block(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        #self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        #self.bn3 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        #out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class Resnet(nn.Module):
    def __init__(self, block, in_channels, kernels, stride=1, padding=1):
        super(Resnet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=stride, padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self.make_layer(block, 64, kernels[0], stride=1)
        self.layer2 = self.make_layer(block, 128, kernels[1], stride=1)
        self.layer3 = self.make_layer(block, 256, kernels[2], stride=1)
        self.layer4 = self.make_layer(block, 512, kernels[3], stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512, 10)  # Output layer for 10 classes
        self.softmax = nn.Softmax(dim=1)  # Softmax layer

    def make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        out = self.softmax(out)  # Apply softmax to get class probabilities
        return out

# Instantiate the Resnet model with in_channels=1 for grayscale input
in_channels = 1  # for grayscale images
kernels = [1, 1, 1, 1]  # Number of blocks in each layer

resnet = Resnet(Resadual_Block, in_channels, kernels).to('cuda')



# New section

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms

# Assuming you have a dataset named 'ds' and a ResNet model defined as 'resnet'

# Define your cross-entropy loss and optimizer
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = Adam(resnet.parameters(), lr=0.001)

# Transform for the images
trans = transforms.ToTensor()

# Number of epochs for training
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    resnet.train()  # Set the model to training mode
    running_loss = 0.0

    for i in tqdm(range(150)):  # Loop over all images in the training dataset
        # Get the image and label
        img = trans(ds['train']['image'][i])
        label = ds['train']['label'][i]

        # Prepare the input tensor and target tensor
        img = img.unsqueeze(0).to('cuda') # Add batch dimension
        target = torch.tensor(label).unsqueeze(0).to('cuda')  # Make sure target is a tensor with batch dimension

        # Forward pass
        out = resnet(img)

        # Calculate loss
        loss = cross_entropy_loss(out, target)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Accumulate the loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = running_loss / len(ds['train']['image'])
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')


100%|██████████| 150/150 [11:25<00:00,  4.57s/it]


Epoch [1/5], Loss: 0.0058


100%|██████████| 150/150 [11:11<00:00,  4.48s/it]


Epoch [2/5], Loss: 0.0055


 74%|███████▍  | 111/150 [08:16<02:53,  4.46s/it]

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms

# Assuming you have a dataset named 'ds' and a ResNet model defined as 'resnet'

# Define your cross-entropy loss and optimizer
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = Adam(resnet.parameters(), lr=0.001)

# Transform for the images
trans = transforms.ToTensor()

# Number of epochs for training
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    resnet.train()  # Set the model to training mode
    running_loss = 0.0

    for i in tqdm(range(150)):  # Loop over all images in the training dataset
        # Get the image and label
        img = trans(ds['train']['image'][i])
        label = ds['train']['label'][i]

        # Prepare the input tensor and target tensor
        img = img.unsqueeze(0).to('cuda') # Add batch dimension
        target = torch.tensor(label).unsqueeze(0).to('cuda')  # Make sure target is a tensor with batch dimension

        # Forward pass
        out = resnet(img)

        # Calculate loss
        loss = cross_entropy_loss(out, target)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Accumulate the loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = running_loss / len(ds['train']['image'])
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')


100%|██████████| 150/150 [11:01<00:00,  4.41s/it]


Epoch [1/5], Loss: 0.0058


100%|██████████| 150/150 [11:02<00:00,  4.42s/it]


Epoch [2/5], Loss: 0.0057


100%|██████████| 150/150 [10:53<00:00,  4.36s/it]


Epoch [3/5], Loss: 0.0057


100%|██████████| 150/150 [10:54<00:00,  4.36s/it]


Epoch [4/5], Loss: 0.0055


100%|██████████| 150/150 [10:54<00:00,  4.36s/it]


Epoch [5/5], Loss: 0.0055
